# Advance Model Exploration

#### Import all the required libraries

In [39]:
# Import libraries
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.corpus import stopwords
import nltk
import warnings
from sklearn.exceptions import ConvergenceWarning
# Suppress warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

#### Read the data into a pandas dataframe

In [15]:
# Read the CSV file into a DataFrame
df = pd.read_csv('../data/original_raw_data.csv')

In [16]:
df.sample(5)

Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
18151       18556      3            0                   0        3      2   
2974         3046      3            0                   3        0      1   
416           422      3            1                   2        0      1   
5763         5927      3            0                   2        1      1   
1573         1608      3            0                   3        0      1   

                                                   tweet  
18151  RT @Yankees: Congrats to Derek Jeter, who will...  
2974   @Darth_Maxwell dude I was pissed but then look...  
416    "@xoxoclauudia: YASSSSSS, ORDERED CHINESE &#12...  
5763          @corey_emanuel bet.... Pay back is a bitch  
1573   &#8220;@ToriJBerman: Shavonne was doing ballet...

In [17]:
# Removing the 'Unnamed: 0' column
df = df.drop('Unnamed: 0', axis=1)

In [18]:
df.sample(5)

count  hate_speech  offensive_language  neither  class  \
7280       3            2                   1        0      0   
8090       6            2                   4        0      1   
10176      3            2                   1        0      0   
756        3            0                   1        2      2   
3763       3            0                   3        0      1   

                                                   tweet  
7280   @white_thunduh my bitch dont want your ugly as...  
8090   Bitches so stupid!!! Yal can't know how I'm bo...  
10176  I didn't forsake all other bitches for my wife...  
756    #RadioFreeNorthwest 218\nCovington reads respo...  
3763                         @KatieFraaancis bitch nigga

### Understanding the Features

    1 - count (numeric - representative of the total number of people who rated this entry)
    2 - hate_speech (numeric - from the total count above)
    3 - offensive_language (numeric - from the total count above)
    4 - neither (numeric - from the total count above)
    5 - class (numeric 0 - offensive , 1 - hate speech or 2 - neither)
    6 - tweet - (text) 

In [19]:
# Initialize lemmatizer, stemmer and TweetTokenizer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
tweet_tokenizer = TweetTokenizer()

In [20]:
# Define stopwords
stop_words = set(stopwords.words('english'))

# Remove 'rt' as it refer to a re-tweet in the tweeet and is non important for our model
stop_words.update(['RT', 'I'])

>The preprocess_text function below **tokenizes** the input text, **lemmatizes and stems the tokens**, **removes stopwords** and **non-alphabetic** tokens. The function then **joins** the processed tokens back into a single string and returns it.

In [21]:
def preprocess_text(text):
    # Tokenize the text
    word_tokens = tweet_tokenizer.tokenize(text)

    # Lemmatize the tokens, stem the tokens, remove stopwords and non-alphabetic tokens
    processed_tokens = [stemmer.stem(lemmatizer.lemmatize(w)) for w in word_tokens if w not in stop_words and w.isalpha()]

    # Join the tokens back into a single string and return it
    return ' '.join(processed_tokens)

In [22]:
# Apply the preprocessing function to the 'tweet' column
df['processed_tweet'] = df['tweet'].apply(preprocess_text)

In [23]:
df

count  hate_speech  offensive_language  neither  class  \
0          3            0                   0        3      2   
1          3            0                   3        0      1   
2          3            0                   3        0      1   
3          3            0                   2        1      1   
4          6            0                   6        0      1   
...      ...          ...                 ...      ...    ...   
24778      3            0                   2        1      1   
24779      3            0                   1        2      2   
24780      3            0                   3        0      1   
24781      6            0                   6        0      1   
24782      3            0                   0        3      2   

                                                   tweet  \
0      !!! RT @mayasolovely: As a woman you shouldn't...   
1      !!!!! RT @mleew17: boy dats cold...tyga dwn ba...   
2      !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...   
3      !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...   
4      !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...   
...                                                  ...   
24778  you's a muthaf***in lie &#8220;@LifeAsKing: @2...   
24779  you've gone and broke the wrong heart baby, an...   
24780  young buck wanna eat!!.. dat nigguh like I ain...   
24781              youu got wild bitches tellin you lies   
24782  ~~Ruffled | Ntac Eileen Dahlia - Beautiful col...   

                                         processed_tweet  
0      as woman complain clean hous man alway take trash  
1         boy dat cold tyga dwn bad cuffin dat hoe place  
2      dawg you ever fuck bitch start cri you confus ...  
3                                       look like tranni  
4      the shit hear might true might faker bitch tol...  
...                                                  ...  
24778  muthaf lie right hi tl trash now mine bibl scr...  
24779    gone broke wrong heart babi drove redneck crazi  
24780  young buck wanna eat dat nigguh like aint fuck...  
24781                     youu got wild bitch tellin lie  
24782  ruffl ntac eileen dahlia beauti color combin p...  

[24783 rows x 7 columns]

### train_test_split 

In [29]:
# Prepare the text data
X_processed = df['processed_tweet']
y = df['class']

X_train_processed, X_test_processed, y_train_processed, y_test_processed = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Advance Model 1

## Logistic Regression

### GridSearchCV - Bag Of Words, Tf-Idf and Hashing 

In [33]:
# Create pipeline with vectorizers
vectorizers = [
    ('count_vectorizer', CountVectorizer()),
    ('tfidf_vectorizer', TfidfVectorizer()),
    ('hashing_vectorizer', HashingVectorizer())
]

# Only use Logistic Regression
classifiers = [
    ('logistic_regression', LogisticRegression(random_state=42))
]

model_results = {}

for vectorizer_name, vectorizer in vectorizers:
    for classifier_name, classifier in classifiers:
        pipe = Pipeline([
            (vectorizer_name, vectorizer),
            (classifier_name, classifier)
        ])
        
        if vectorizer_name != 'hashing_vectorizer':
            param_grid = {
                'logistic_regression__C': [0.1, 1, 10],
                f'{vectorizer_name}__ngram_range': [(1, 1), (1, 2)],
                f'{vectorizer_name}__min_df': [1, 2, 5],
                f'{vectorizer_name}__stop_words': [None, 'english'],
                f'{vectorizer_name}__max_features': [None, 500, 1000]
            }
        else: # no params for HashingVectorizer
            param_grid = {
                'logistic_regression__C': [0.1, 1, 10]
            }

        model_key = f'{classifier_name.replace("_", " ").title()} {vectorizer_name.replace("_", " ").title()}'
        model_results[model_key] = {'best_params': None, 'best_score': 0, 'fit_time': 0}

        grid = GridSearchCV(pipe, param_grid, cv=5, verbose=1)
        start_time = time.time()
        grid.fit(X_train_processed, y_train_processed)
        elapsed_time = time.time() - start_time

        if grid.best_score_ > model_results[model_key]['best_score']:
            model_results[model_key]['best_params'] = grid.best_params_
            model_results[model_key]['best_score'] = grid.best_score_
            model_results[model_key]['fit_time'] = elapsed_time

results_df = pd.DataFrame(model_results).T

results_df

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Fitting 5 folds for each of 108 candidates, totalling 540 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits


best_params best_score    fit_time
Logistic Regression Count Vectorizer    {'count_vectorizer__max_features': None, 'count_vectorizer__min_df': 2, 'count_vectorizer__ngram_range': (1, 2), 'count_vectorizer__stop_words': None, 'logistic_regression__C': 0.1}   0.904267  296.870057
Logistic Regression Tfidf Vectorizer     {'logistic_regression__C': 10, 'tfidf_vectorizer__max_features': None, 'tfidf_vectorizer__min_df': 2, 'tfidf_vectorizer__ngram_range': (1, 2), 'tfidf_vectorizer__stop_words': None}   0.898366  291.928122
Logistic Regression Hashing Vectorizer                                                                                                                                                          {'logistic_regression__C': 1}   0.905175  257.981519

# Advance Model 2

## Decision Tree

### GridSearchCV - Bag Of Words, Tf-Idf, Hashing and Binary vectorazation

In [37]:
# Create pipeline with vectorizers and classifiers
vectorizers = [
    ('count_vectorizer', CountVectorizer()),
    ('tfidf_vectorizer', TfidfVectorizer()),
    ('hashing_vectorizer', HashingVectorizer(norm=None)),
    ('binary_vectorizer', CountVectorizer(binary=True))
]

classifiers = [
    ('decision_tree', DecisionTreeClassifier(random_state=42))
]

model_results = {}

for vectorizer_name, vectorizer in vectorizers:
    for classifier_name, classifier in classifiers:
        pipe = Pipeline([
            (vectorizer_name, vectorizer),
            (classifier_name, classifier)
        ])

        if classifier_name == 'decision_tree':
            param_grid = {
                'decision_tree__max_depth': [5, 10, 15],
                f'{vectorizer_name}__ngram_range': [(1, 1), (1, 2)],
                f'{vectorizer_name}__stop_words': [None, 'english']
            }
            if vectorizer_name != 'hashing_vectorizer':
                param_grid[f'{vectorizer_name}__min_df'] = [1, 2, 5]
                param_grid[f'{vectorizer_name}__max_features'] = [None, 500, 1000]

        
        model_key = f'{classifier_name.replace("_", " ").title()} {vectorizer_name.replace("_", " ").title()}'
        model_results[model_key] = {'best_params': None, 'best_score': 0, 'fit_time': 0}

        grid = GridSearchCV(pipe, param_grid, cv=5, verbose=1)
        start_time = time.time()
        grid.fit(X_train_processed, y_train_processed)
        elapsed_time = time.time() - start_time
        
        if grid.best_score_ > model_results[model_key]['best_score']:
            model_results[model_key]['best_params'] = grid.best_params_
            model_results[model_key]['best_score'] = grid.best_score_
            model_results[model_key]['fit_time'] = elapsed_time

results_df = pd.DataFrame(model_results).T

results_df

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Fitting 5 folds for each of 108 candidates, totalling 540 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 108 candidates, totalling 540 fits


best_params best_score    fit_time
Decision Tree Count Vectorizer        {'count_vectorizer__max_features': None, 'count_vectorizer__min_df': 1, 'count_vectorizer__ngram_range': (1, 2), 'count_vectorizer__stop_words': None, 'decision_tree__max_depth': 15}   0.890346  125.677383
Decision Tree Tfidf Vectorizer        {'decision_tree__max_depth': 15, 'tfidf_vectorizer__max_features': None, 'tfidf_vectorizer__min_df': 2, 'tfidf_vectorizer__ngram_range': (1, 2), 'tfidf_vectorizer__stop_words': None}   0.889892    148.8977
Decision Tree Hashing Vectorizer                                                                         {'decision_tree__max_depth': 15, 'hashing_vectorizer__ngram_range': (1, 2), 'hashing_vectorizer__stop_words': None}    0.89085   93.428398
Decision Tree Binary Vectorizer   {'binary_vectorizer__max_features': None, 'binary_vectorizer__min_df': 1, 'binary_vectorizer__ngram_range': (1, 2), 'binary_vectorizer__stop_words': None, 'decision_tree__max_depth': 15}   0.891254  127.905169

# Advance Model 3

## Random Forest

### GridSearchCV - Bag Of Words, Tf-Idf, Hashing and Binary vectorazation

In [ ]:
# Create pipeline with vectorizers and classifiers
vectorizers = [
    ('count_vectorizer', CountVectorizer()),
    ('tfidf_vectorizer', TfidfVectorizer()),
    ('hashing_vectorizer', HashingVectorizer(norm=None)),
    ('binary_vectorizer', CountVectorizer(binary=True))
]

classifiers = [
    ('random_forest', RandomForestClassifier(n_estimators=100, random_state=42))
]

model_results = {}

for vectorizer_name, vectorizer in vectorizers:
    for classifier_name, classifier in classifiers:
        pipe = Pipeline([
            (vectorizer_name, vectorizer),
            (classifier_name, classifier)
        ])

        if classifier_name == 'random_forest':
            param_grid = {
                'random_forest__n_estimators': [100, 200, 300],
                'random_forest__max_depth': [5, 10, 15],
                f'{vectorizer_name}__ngram_range': [(1, 1), (1, 2)],
                f'{vectorizer_name}__min_df': [1, 2, 5],
                f'{vectorizer_name}__stop_words': [None, 'english'],
                f'{vectorizer_name}__max_features': [None, 500, 1000]
            }
            if vectorizer_name != 'hashing_vectorizer':
                param_grid[f'{vectorizer_name}__min_df'] = [1, 2, 5]
                param_grid[f'{vectorizer_name}__max_features'] = [None, 500, 1000]

        model_key = f'{classifier_name.replace("_", " ").title()} {vectorizer_name.replace("_", " ").title()}'
        model_results[model_key] = {'best_params': None, 'best_score': 0, 'fit_time': 0}

        grid = GridSearchCV(pipe, param_grid, cv=5, verbose=1)
        start_time = time.time()
        grid.fit(X_train_processed, y_train_processed)
        elapsed_time = time.time() - start_time
        
        if grid.best_score_ > model_results[model_key]['best_score']:
            model_results[model_key]['best_params'] = grid.best_params_
            model_results[model_key]['best_score'] = grid.best_score_
            model_results[model_key]['fit_time'] = elapsed_time

results_df = pd.DataFrame(model_results).T

results_df

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


# Advance Model 4

## Support Vector Machine

### GridSearchCV - Bag Of Words, Tf-Idf, Hashing and Binary vectorazation

In [ ]:
# Create pipeline with vectorizers and classifiers
vectorizers = [
    ('count_vectorizer', CountVectorizer()),
    ('tfidf_vectorizer', TfidfVectorizer()),
    ('hashing_vectorizer', HashingVectorizer(norm=None)),
    ('binary_vectorizer', CountVectorizer(binary=True))
]

classifiers = [
    ('svc', SVC(random_state=42))
]

model_results = {}

for vectorizer_name, vectorizer in vectorizers:
    for classifier_name, classifier in classifiers:
        pipe = Pipeline([
            (vectorizer_name, vectorizer),
            (classifier_name, classifier)
        ])

        if classifier_name == 'svc':
            param_grid = {
                'svc__C': [0.1, 1, 10],
                'svc__kernel': ['linear', 'rbf'],
                f'{vectorizer_name}__ngram_range': [(1, 1), (1, 2)],
                f'{vectorizer_name}__min_df': [1, 2, 5],
                f'{vectorizer_name}__stop_words': [None, 'english'],
                f'{vectorizer_name}__max_features': [None, 500, 1000]
            }
            if vectorizer_name != 'hashing_vectorizer':
                param_grid[f'{vectorizer_name}__max_features'] = [None, 500, 1000]

        model_key = f'{classifier_name.replace("_", " ").title()} {vectorizer_name.replace("_", " ").title()}'
        model_results[model_key] = {'best_params': None, 'best_score': 0, 'fit_time': 0}

        grid = GridSearchCV(pipe, param_grid, cv=5, verbose=1)
        start_time = time.time()
        grid.fit(X_train_processed, y_train_processed)
        elapsed_time = time.time() - start_time
        
        if grid.best_score_ > model_results[model_key]['best_score']:
            model_results[model_key]['best_params'] = grid.best_params_
            model_results[model_key]['best_score'] = grid.best_score_
            model_results[model_key]['fit_time'] = elapsed_time

results_df = pd.DataFrame(model_results).T

results_df

# Advance Model 5

## K Neigbors

### GridSearchCV - Bag Of Words, Tf-Idf, Hashing and Binary vectorazation

In [ ]:
# Create pipeline with vectorizers and classifiers
vectorizers = [
    ('count_vectorizer', CountVectorizer()),
    ('tfidf_vectorizer', TfidfVectorizer()),
    ('hashing_vectorizer', HashingVectorizer(non_negative=True, norm=None)),
    ('binary_vectorizer', CountVectorizer(binary=True))
]

classifiers = [
    ('k_neighbors', KNeighborsClassifier())
]

model_results = {}

for vectorizer_name, vectorizer in vectorizers:
    for classifier_name, classifier in classifiers:
        pipe = Pipeline([
            (vectorizer_name, vectorizer),
            (classifier_name, classifier)
        ])

        if classifier_name == 'k_neighbors':
            param_grid = {
                'k_neighbors__n_neighbors': [3, 5, 7],
                f'{vectorizer_name}__ngram_range': [(1, 1), (1, 2)],
                f'{vectorizer_name}__min_df': [1, 2, 5],
                f'{vectorizer_name}__stop_words': [None, 'english'],
                f'{vectorizer_name}__max_features': [None, 500, 1000]
            }
            if vectorizer_name != 'hashing_vectorizer':
                param_grid[f'{vectorizer_name}__max_features'] = [None, 500, 1000]
        

        model_key = f'{classifier_name.replace("_", " ").title()} {vectorizer_name.replace("_", " ").title()}'
        model_results[model_key] = {'best_params': None, 'best_score': 0, 'fit_time': 0}

        grid = GridSearchCV(pipe, param_grid, cv=5, verbose=1)
        start_time = time.time()
        grid.fit(X_train_processed, y_train_processed)
        elapsed_time = time.time() - start_time
        
        if grid.best_score_ > model_results[model_key]['best_score']:
            model_results[model_key]['best_params'] = grid.best_params_
            model_results[model_key]['best_score'] = grid.best_score_
            model_results[model_key]['fit_time'] = elapsed_time

results_df = pd.DataFrame(model_results).T

results_df

# Advance Model 6

## XGB

### GridSearchCV - Bag Of Words, Tf-Idf, Hashing and Binary vectorazation

In [ ]:
# Create pipeline with vectorizers and classifiers
vectorizers = [
    ('count_vectorizer', CountVectorizer()),
    ('tfidf_vectorizer', TfidfVectorizer()),
    ('hashing_vectorizer', HashingVectorizer(non_negative=True, norm=None)),
    ('binary_vectorizer', CountVectorizer(binary=True))
]

classifiers = [
    ('decision_tree', DecisionTreeClassifier(random_state=42)),
    ('random_forest', RandomForestClassifier(random_state=42)),
    ('svc', SVC(random_state=42)),
    ('k_neighbors', KNeighborsClassifier()),
    ('xgb', XGBClassifier(random_state=42))
]

model_results = {}

for vectorizer_name, vectorizer in vectorizers:
    for classifier_name, classifier in classifiers:
        pipe = Pipeline([
            (vectorizer_name, vectorizer),
            (classifier_name, classifier)
        ])

        if classifier_name == 'decision_tree':
            param_grid = {
                'decision_tree__max_depth': [5, 10, 15],
                f'{vectorizer_name}__ngram_range': [(1, 1), (1, 2)],
                f'{vectorizer_name}__min_df': [1, 2, 5],
                f'{vectorizer_name}__stop_words': [None, 'english'],
                f'{vectorizer_name}__max_features': [None, 500, 1000]
            }
        elif classifier_name == 'random_forest':
            param_grid = {
                'random_forest__n_estimators': [100, 200, 300],
                'random_forest__max_depth': [5, 10, 15],
                f'{vectorizer_name}__ngram_range': [(1, 1), (1, 2)],
                f'{vectorizer_name}__min_df': [1, 2, 5],
                f'{vectorizer_name}__stop_words': [None, 'english'],
                f'{vectorizer_name}__max_features': [None, 500, 1000]
            }
        elif classifier_name == 'svc':
            param_grid = {
                'svc__C': [0.1, 1, 10],
                'svc__kernel': ['linear', 'rbf'],
                f'{vectorizer_name}__ngram_range': [(1, 1), (1, 2)],
                f'{vectorizer_name}__min_df': [1, 2, 5],
                f'{vectorizer_name}__stop_words': [None, 'english'],
                f'{vectorizer_name}__max_features': [None, 500, 1000]
            }
        elif classifier_name == 'k_neighbors':
            param_grid = {
                'k_neighbors__n_neighbors': [3, 5, 7],
                f'{vectorizer_name}__ngram_range': [(1, 1), (1, 2)],
                f'{vectorizer_name}__min_df': [1, 2, 5],
                f'{vectorizer_name}__stop_words': [None, 'english'],
                f'{vectorizer_name}__max_features': [None, 500, 1000]
            }
        elif classifier_name == 'xgb':
            param_grid = {
                'xgb__n_estimators': [100, 200, 300],
                'xgb__max_depth': [3, 5, 7],
                'xgb__learning_rate': [0.1, 0.01],
                f'{vectorizer_name}__ngram_range': [(1, 1), (1, 2)],
                f'{vectorizer_name}__min_df': [1, 2, 5],
                f'{vectorizer_name}__stop_words': [None, 'english'],
                f'{vectorizer_name}__max_features': [None, 500, 1000]
            }

        model_key = f'{classifier_name.replace("_", " ").title()} {vectorizer_name.replace("_", " ").title()}'
        model_results[model_key] = {'best_params': None, 'best_score': 0, 'fit_time': 0}

        grid = GridSearchCV(pipe, param_grid, cv=5, verbose=1)
        start_time = time.time()
        grid.fit(X_train_processed, y_train_processed)
        elapsed_time = time.time() - start_time
        
        if grid.best_score_ > model_results[model_key]['best_score']:
            model_results[model_key]['best_params'] = grid.best_params_
            model_results[model_key]['best_score'] = grid.best_score_
            model_results[model_key]['fit_time'] = elapsed_time

results_df = pd.DataFrame(model_results).T

results_df

In [ ]:
# Create pipeline with vectorizers and classifiers
vectorizers = [
    ('count_vectorizer', CountVectorizer()),
    ('tfidf_vectorizer', TfidfVectorizer()),
    ('hashing_vectorizer', HashingVectorizer(non_negative=True, norm=None)),
    ('binary_vectorizer', CountVectorizer(binary=True))
]

classifiers = [
    ('decision_tree', DecisionTreeClassifier(random_state=42)),
    ('random_forest', RandomForestClassifier(random_state=42)),
    ('svc', SVC(random_state=42)),
    ('k_neighbors', KNeighborsClassifier()),
    ('xgb', XGBClassifier(random_state=42))
]

model_results = {}

for vectorizer_name, vectorizer in vectorizers:
    for classifier_name, classifier in classifiers:
        pipe = Pipeline([
            (vectorizer_name, vectorizer),
            (classifier_name, classifier)
        ])

        if classifier_name == 'decision_tree':
            param_grid = {
                'decision_tree__max_depth': [5, 10, 15],
                f'{vectorizer_name}__ngram_range': [(1, 1), (1, 2)],
                f'{vectorizer_name}__min_df': [1, 2, 5],
                f'{vectorizer_name}__stop_words': [None, 'english'],
                f'{vectorizer_name}__max_features': [None, 500, 1000]
            }
        elif classifier_name == 'random_forest':
            param_grid = {
                'random_forest__n_estimators': [100, 200, 300],
                'random_forest__max_depth': [5, 10, 15],
                f'{vectorizer_name}__ngram_range': [(1, 1), (1, 2)],
                f'{vectorizer_name}__min_df': [1, 2, 5],
                f'{vectorizer_name}__stop_words': [None, 'english'],
                f'{vectorizer_name}__max_features': [None, 500, 1000]
            }
        elif classifier_name == 'svc':
            param_grid = {
                'svc__C': [0.1, 1, 10],
                'svc__kernel': ['linear', 'rbf'],
                f'{vectorizer_name}__ngram_range': [(1, 1), (1, 2)],
                f'{vectorizer_name}__min_df': [1, 2, 5],
                f'{vectorizer_name}__stop_words': [None, 'english'],
                f'{vectorizer_name}__max_features': [None, 500, 1000]
            }
        elif classifier_name == 'k_neighbors':
            param_grid = {
                'k_neighbors__n_neighbors': [3, 5, 7],
                f'{vectorizer_name}__ngram_range': [(1, 1), (1, 2)],
                f'{vectorizer_name}__min_df': [1, 2, 5],
                f'{vectorizer_name}__stop_words': [None, 'english'],
                f'{vectorizer_name}__max_features': [None, 500, 1000]
            }
        elif classifier_name == 'xgb':
            param_grid = {
                'xgb__n_estimators': [100, 200, 300],
                'xgb__max_depth': [3, 5, 7],
                'xgb__learning_rate': [0.1, 0.01],
                f'{vectorizer_name}__ngram_range': [(1, 1), (1, 2)],
                f'{vectorizer_name}__min_df': [1, 2, 5],
                f'{vectorizer_name}__stop_words': [None, 'english'],
                f'{vectorizer_name}__max_features': [None, 500, 1000]
            }

        model_key = f'{classifier_name.replace("_", " ").title()} {vectorizer_name.replace("_", " ").title()}'
        model_results[model_key] = {'best_params': None, 'best_score': 0, 'fit_time': 0}

        grid = GridSearchCV(pipe, param_grid, cv=5, verbose=1)
        start_time = time.time()
        grid.fit(X_train_processed, y_train_processed)
        elapsed_time = time.time() - start_time
        
        if grid.best_score_ > model_results[model_key]['best_score']:
            model_results[model_key]['best_params'] = grid.best_params_
            model_results[model_key]['best_score'] = grid.best_score_
            model_results[model_key]['fit_time'] = elapsed_time

results_df = pd.DataFrame(model_results).T

results_df